<a href="https://colab.research.google.com/github/Valdi35/Detect-HFT-using-machine-learning-models/blob/main/Challenge_AMF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Le but du challenge est de construire un modèle de machine learning pouvant prédire les trois catégories de traders (HFT, non HFT et MIX). Au delà d'obtenir une meilleure accuracy, le second objectif consiste à choisir les variables les plus pertinentes.

In [1]:
#Install spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz
!tar -xvf spark-3.0.3-bin-hadoop2.7.tgz
!pip install -q findspark

spark-3.0.3-bin-hadoop2.7/
spark-3.0.3-bin-hadoop2.7/NOTICE
spark-3.0.3-bin-hadoop2.7/kubernetes/
spark-3.0.3-bin-hadoop2.7/kubernetes/tests/
spark-3.0.3-bin-hadoop2.7/kubernetes/tests/worker_memory_check.py
spark-3.0.3-bin-hadoop2.7/kubernetes/tests/py_container_checks.py
spark-3.0.3-bin-hadoop2.7/kubernetes/tests/pyfiles.py
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/spark/
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/spark/entrypoint.sh
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/R/
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/R/Dockerfile
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/python/
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/python/Dockerfile
spark-3.0.3-bin-hadoop2.7/kubernetes/dockerfiles/spark/Dockerfile
spark-3.0.3-bin-hadoop2.7/jars/
spark-3.0.3-bin-hadoop2.7/jars/jacks

In [31]:
#Download libraries
import numpy as np
import pandas as pd
import io
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"
import findspark
findspark.init()
from google.colab import files
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import isnan, when, count, col, lit, trim, avg, ceil
from pyspark.sql.types import StringType
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pyspark.sql import functions as F

In [3]:
#Data
uploaded = files.upload()

Saving AMF_test_X.csv to AMF_test_X.csv
Saving AMF_train_X.csv to AMF_train_X.csv
Saving AMF_train_Y.csv to AMF_train_Y.csv


In [4]:
sc = SparkSession.builder.master("local[*]").getOrCreate()

In [16]:
#Read data
df = sc.read.csv('AMF_train_X.csv', inferSchema=True, header=True)

#Lower columns names
for col in df.columns:
    df = df.withColumnRenamed(col, col.lower())

In [14]:
y_train = sc.read.csv('AMF_train_Y.csv', inferSchema=True, header=True)
for col in y_train.columns:
    y_train = y_train.withColumnRenamed(col, col.lower())

In [17]:
#Join the two df on trader id
data = df.join(y_train, on=("trader"))

List of features
x_Train and x_test data exhibit in their rows the same 35 features calculated for a given market player i on a certain stock Isin_iIsin 
i
​	
  (whose identification code is an Isin) and a specific trading date tt :


1 - number of trading venues on which the market player trades ;
from all trading venues, statistics over the number of trades observed per second : #2 the mean, and,
the max

2 - statistics over the observed time delta between two trades on the trading venue TV_1[1] :
min,
median,
min

3 - statistics over the observed time delta between two trades occurring on trading venue TV_1 and then on trading venue TV_2 :
min,
median,
min

4 - statistics over the observed time delta between two trades occurring on trading venue TV_1 and then on trading venue TV_3 :
min,
median,
min

5 - statistics over the observed time delta between two trades occurring on trading venue TV_1 and then on trading venue TV_4 :
min,
median,
min

6 - from all trading venues, number of seconds during the trading day where at least one trade of the market player i is observed
on trading venue TV_1, three ratios between the number of all types of events[2] sent to the LOB and :
the number of trades (OTR),
the number of cancellation-type event (OCR),
the number of modification-type event (OMR)

7 - on trading venue TV_1, statistics over the observed time delta between two all-type events sent :
min,
mean,
10th percentile,
1st quartile,
median,
3rd quartile,
90th percentile,
max

8 - on trading venue TV_1, statistics over the observed lifetime of cancelled orders :
min,
mean,
10th percentile,
1st quartile,
median,
3rd quartile,
90th percentile,
max

The features above are not detailed in the same order as in the challenge files.



[1] TV_1 is the trading venue with the highest volume traded
[2] Events include both the transactions and the messages that market players can send to the LOB: new order, order modification or order cancellation.

In [21]:
print(data.count())
print(data.columns)

105782
['trader', 'index', 'share', 'day', 'otr', 'ocr', 'omr', 'min_time_two_events', 'mean_time_two_events', '10_p_time_two_events', 'med_time_two_events', '25_p_time_two_events', '75_p_time_two_events', '90_p_time_two_events', 'max_time_two_events', 'min_lifetime_cancel', 'mean_lifetime_cancel', '10_p_lifetime_cancel', 'med_lifetime_cancel', '25_p_lifetime_cancel', '75_p_lifetime_cancel', '90_p_lifetime_cancel', 'max_lifetime_cancel', 'nbtradevenuemic', 'maxnbtradesbysecond', 'meannbtradesbysecond', 'min_dt_tv1', 'mean_dt_tv1', 'med_dt_tv1', 'min_dt_tv1_tv2', 'mean_dt_tv1_tv2', 'med_dt_tv1_tv2', 'min_dt_tv1_tv3', 'mean_dt_tv1_tv3', 'med_dt_tv1_tv3', 'min_dt_tv1_tv4', 'mean_dt_tv1_tv4', 'med_dt_tv1_tv4', 'nbsecondwithatleatonetrade', 'type']


In [22]:
#head 10 first value
print(data.show(10))

+---------+-----+--------+-------+------------------+------------------+------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------------+---------------+-------------------+--------------------+----------+------------------+-----------+--------------+---------------+--------------+--------------+---------------+--------------+--------------+---------------+--------------+--------------------------+-------+
|   trader|index|   share|    day|               otr|               ocr|               omr|min_time_two_events|mean_time_two_events|10_p_time_two_events|med_time_two_events|25_p_time_two_events|75_p_time_two_events|90_p_time_two_events|max_time_two_events|min_lifetime_cancel|mean_lifetime_cancel|10_p_life

In [23]:
print(data.printSchema())

root
 |-- trader: string (nullable = true)
 |-- index: integer (nullable = true)
 |-- share: string (nullable = true)
 |-- day: string (nullable = true)
 |-- otr: double (nullable = true)
 |-- ocr: double (nullable = true)
 |-- omr: double (nullable = true)
 |-- min_time_two_events: double (nullable = true)
 |-- mean_time_two_events: double (nullable = true)
 |-- 10_p_time_two_events: double (nullable = true)
 |-- med_time_two_events: double (nullable = true)
 |-- 25_p_time_two_events: double (nullable = true)
 |-- 75_p_time_two_events: double (nullable = true)
 |-- 90_p_time_two_events: double (nullable = true)
 |-- max_time_two_events: double (nullable = true)
 |-- min_lifetime_cancel: double (nullable = true)
 |-- mean_lifetime_cancel: double (nullable = true)
 |-- 10_p_lifetime_cancel: double (nullable = true)
 |-- med_lifetime_cancel: double (nullable = true)
 |-- 25_p_lifetime_cancel: double (nullable = true)
 |-- 75_p_lifetime_cancel: double (nullable = true)
 |-- 90_p_lifetime_

In [24]:
data = data.drop("index")

In [32]:
#Remove columns with null values more than a threeshold (0.2)
agg_row = data.select([(count(when(isnan(c) | F.col(c).isNull(), c))/data.count()).alias(c) for c in data.columns if c not in
                       {'day', 'trader', 'type', 'share'}]).collect()

In [35]:
agg_dict_list = [row.asDict() for row in agg_row]
agg_dict = agg_dict_list[0]

In [37]:
print(agg_dict)

{'otr': 0.015910079219526953, 'ocr': 0.0, 'omr': 0.4500954793821255, 'min_time_two_events': 0.0, 'mean_time_two_events': 0.0, '10_p_time_two_events': 0.0, 'med_time_two_events': 0.0, '25_p_time_two_events': 0.0, '75_p_time_two_events': 0.0, '90_p_time_two_events': 0.0, 'max_time_two_events': 0.0, 'min_lifetime_cancel': 0.0, 'mean_lifetime_cancel': 0.0, '10_p_lifetime_cancel': 0.0, 'med_lifetime_cancel': 0.0, '25_p_lifetime_cancel': 0.0, '75_p_lifetime_cancel': 0.0, '90_p_lifetime_cancel': 0.0, 'max_lifetime_cancel': 0.0, 'nbtradevenuemic': 0.0, 'maxnbtradesbysecond': 0.0, 'meannbtradesbysecond': 0.0, 'min_dt_tv1': 0.04003516666351553, 'mean_dt_tv1': 0.04003516666351553, 'med_dt_tv1': 0.04003516666351553, 'min_dt_tv1_tv2': 0.23799890340511617, 'mean_dt_tv1_tv2': 0.23799890340511617, 'med_dt_tv1_tv2': 0.23799890340511617, 'min_dt_tv1_tv3': 0.2387457223346127, 'mean_dt_tv1_tv3': 0.2387457223346127, 'med_dt_tv1_tv3': 0.2387457223346127, 'min_dt_tv1_tv4': 0.23773420808833262, 'mean_dt_tv1_t

In [38]:
col_null = list({i for i in agg_dict if agg_dict[i] > 0.2})
print(col_null)
data = data.drop(*col_null)

['omr', 'mean_dt_tv1_tv2', 'med_dt_tv1_tv3', 'min_dt_tv1_tv4', 'med_dt_tv1_tv4', 'mean_dt_tv1_tv3', 'min_dt_tv1_tv3', 'min_dt_tv1_tv2', 'med_dt_tv1_tv2', 'mean_dt_tv1_tv4']


Cross tables 